# Data Export
1. Copies cocktails_db.pkl into api/src <i>(to be deployed with the next lambda)</i>
2. Uploads ingredients_db.pkl to algolia

# 1. Copies cocktails_db.pkl into api/src

In [18]:
import os
os.system('cp cocktails_db.pkl ../../src/')

0

# 2. Uploads ingredients_db.pkl to Algolia

In [19]:
import pickle
from algoliasearch.search_client import SearchClient
client = SearchClient.create('XDZGBI1FCU', '81aed0e97eb67f4d0c54548d18fc2fff')
index = client.init_index('ingredients')
index.clear_objects()


In [20]:
import boto3

icdf = pickle.load(open('ingredients_db.pkl', 'rb'))
print('{} total ingredients'.format(len(icdf)))

# Remove ingredients without an image in S3
bucket_name='wedgecocktail-ingredient-images'
session = boto3.Session(profile_name='default')
client = session.client('s3', region_name='us-west-2')
s3_images = [key['Key'].replace('.png', '') for key in client.list_objects(Bucket=bucket_name)['Contents']]
icdf = icdf.loc[icdf.objectID.isin(s3_images)]

# Remove blacklisted category_ids
blacklist = ['water', 'ice']
icdf = icdf.loc[~icdf.objectID.isin(blacklist)]

to_upload = icdf.to_dict(orient='records')

print('{} ingredients to upload'.format(len(to_upload)))

553 total ingredients
242 ingredients to upload


In [21]:
index.save_objects(to_upload)
index.set_settings({"searchableAttributes": ["name"]})

# Scratch Cocktail DF Exploration

In [2]:
import pickle
import pandas as pd

In [6]:
cdf = pickle.load(open('../../src/cocktails_db.pkl', 'rb'))

In [7]:
len(cdf)

2980

In [16]:
to_add = [
    {
        'name':'Screwdriver',
        'image_url':'https://us.inshaker.com/uploads/cocktail/hires/52/1629735771-%D0%9E%D1%82%D0%B2%D0%B5%D1%80%D1%82%D0%BA%D0%B0.jpg',
        'serving_container':'Collins Glass',
        'instructions':'STIR (preferably with a screwdriver) all ingredients with ice and strain into ice-filled glass.',
        'objectID':'screwdriver',
        'num_ingredients':2,
        'ingredients':[{'quantity':'1 1/2 fl oz','name':'vodka'},{'quantity':'5 fl oz','name':'orange juice'}],
        'ingredient_categories':['vodka','orange-juice']
    },
        {
        'name':'Vodka Soda',
        'image_url':'https://kinkybeverages.com/wp-content/uploads/2020/07/Vodka-Soda.png',
        'serving_container':'Collins Glass',
        'instructions':'POUR ingredients into ice-filled glass and lightly stir (so as not to lose fizz). Garnish with lime.',
        'objectID':'vodka-soda',
        'num_ingredients':2,
        'ingredients':[{'quantity':'2 fl oz','name':'vodka'},{'quantity':'5 fl oz','name':'soda water'}],
        'ingredient_categories':['vodka','soda-water']
    },
    {
        'name':'Vodka Tonic',
        'image_url':'https://kinkybeverages.com/wp-content/uploads/2020/07/Vodka-Soda.png',
        'serving_container':'Collins Glass',
        'instructions':'POUR ingredients into ice-filled glass and lightly stir (so as not to lose fizz). Garnish with lime.',
        'objectID':'vodka-tonic',
        'num_ingredients':2,
        'ingredients':[{'quantity':'2 fl oz','name':'vodka'},{'quantity':'5 fl oz','name':'tonic water'}],
        'ingredient_categories':['vodka','tonic-water']
    }
]

In [17]:
adf = pd.DataFrame(to_add)

In [18]:
len(cdf)

2980

In [22]:
len(pd.concat([cdf, adf]))

2983

In [23]:
cdf = pd.concat([cdf, adf])

In [24]:
pickle.dump(cdf, open('cocktails_db.pkl','wb'))